In [1]:
from ucimlrepo import fetch_ucirepo 
import os
import certifi
import ssl
import pandas as pd
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn import svm, datasets
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

# I need this for my python to work
os.environ['SSL_CERT_FILE'] = '/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/certifi/cacert.pem'

adult = fetch_ucirepo(id=2) 
  
# data (as pandas dataframes) 
X = adult.data.features 
y = adult.data.targets 
  
# metadata 
print(adult.metadata) 
  
# variable information 
print(adult.variables) 


ConnectionError: Error connecting to server

Task 2

In [ ]:
adult.data.features.head()

In [ ]:
adult.data.features.info()

In [ ]:
adult.data.features.describe()

In [ ]:
adult.data.features.shape

In [ ]:
adult.data.targets.head()

In [ ]:
concat = pd.concat((X, y), axis=1)
concat

In [ ]:
# Make histogram bigger
plt.figure(figsize=(12, 8)) 

adult.data.features.hist()

#plt.xticks(rotation=45)

#plt.tight_layout()

plt.show()

Task 3

In [ ]:
def detect_question_mark(x):
    if pd.isna(x):  
        return False
    return str(x).strip() == '?'

question_mark = X.map(detect_question_mark)

question_sum = question_mark.sum()

question_sum


Task 4

In [ ]:
X_copy = X.copy()

# Replace '?' with NaN in the copy of the DataFrame
X_copy = X_copy.replace('?', np.nan)

missing_count = X_copy.isna().sum()
print(missing_count)

In [ ]:
X_copy.info()

Task 5

In [ ]:
# Split dataframe into numerical and categorial for the pipeline
numerical_cols = X.select_dtypes(include=['int64']).columns
categorical_cols = X.select_dtypes(include=['object']).columns

In [ ]:
# Fill in missing numerical values with SimpleImputer
# Then scale with StandardScaler

num_imp = SimpleImputer(missing_values=np.nan, strategy='mean')
Xn_imp = num_imp.fit_transform(X[numerical_cols])


scaler_n = StandardScaler()
scaled_n = scaler_n.fit_transform(Xn_imp)

scaled_n

In [ ]:
# Fill in most frequent values with SimpleImputer
# Then encode with OneHotEncoder

cat_imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
Xc_imp = cat_imp.fit_transform(X[categorical_cols])

cat_enc = OneHotEncoder(handle_unknown='ignore')
Xc_encoded = cat_enc.fit_transform(Xc_imp)
Xc_encoded

In [ ]:
# Combine the numerical and categorial columns into one Dataframe
scaled_n_df = pd.DataFrame(scaled_n)


Xc_encoded_df = pd.DataFrame(Xc_encoded.toarray())


X_prepared = pd.concat([scaled_n_df, Xc_encoded_df], axis=1)


X_prepared.shape


Task 6

In [ ]:
y.value_counts()

In [ ]:
Task 7

In [ ]:
y_clean = y.copy()
y_clean.replace({'>50K.': '>50K', '<=50K.': '<=50K'}, inplace=True)
cleaned_value_counts = y_clean.value_counts()
cleaned_value_counts

Task 8

In [ ]:
# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(
    X_prepared, y_clean, test_size=0.2, random_state=42
)

# Print the shapes of the splits
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)


Task 9

In [ ]:
# Train the model
model_svm = SVC(kernel='poly', C=0.1, gamma=1)

model_svm.fit(X_train, y_train.values.ravel())

# Test model and get classisification report

y_pred = model_svm.predict(X_test)

report = classification_report(y_test, y_pred)

report

In [ ]:
# Create Confusion Matrix

cm = confusion_matrix(y_test, y_pred)

ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model_svm.classes_)

Task 10

In [ ]:
# Split the data into training, validation and testing sets (60% training, 20% validation, and 20% testing)

X_train, X_validation_test, y_train, y_validation_test = train_test_split(X_prepared, y_clean, test_size=0.4, random_state=42)

X_validation, X_test, y_validation, y_test = train_test_split(X_validation_test, y_validation_test, test_size=0.5, random_state=42)

print(X_train.shape, y_train.shape, X_validation.shape, y_validation.shape, X_test.shape, y_test.shape)

In [ ]:
# Perform GridSearchCV

svm_parameters = {
    'kernel': ['rbf'],
    'C': [0.01, 0.1, 1, 10],
    'gamma': [0.01, 1, 10]
}

svm = SVC()
svm_gs = GridSearchCV(estimator=svm, param_grid=svm_parameters)

svm_gs.fit(X_train.iloc[:10000], y_train.iloc[:10000].values.ravel())

svm_winner = svm_gs.best_estimator_
svm_winner.score(X_validation, y_validation)



In [ ]:
# Print the parameters

best_parameters = svm_winner.get_params()
print(best_parameters)

In [ ]:
# Print the specific parameters of interest
print(f"Kernel: {svm_winner.kernel}")
print(f"C: {svm_winner.C}")
print(f"Gamma: {svm_winner.gamma}")
